# 1. Import libraries and load data from database.
Import Python libraries

Load dataset from database with read_sql_table

Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

# 2. Write a tokenization function to process your text data


In [3]:
def tokenize(text):
    """
    inputs:
    messages
       
    Returns:
    list of words into numbers of same meaning
    """
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

# 3. Build a machine learning pipeline
This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may find the MultiOutputClassifier helpful for predicting multiple target variables.

In [4]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# 4. Train pipeline
Split data into train and test sets
Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(13)
pipe.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000194B88BDEE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

# 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's classification_report on each.

In [6]:
def eval_metrics(ArrayL, ArrayP, col_names):
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics

In [7]:
# Calculate evaluation metrics for training set
Y_train_pred = pipe.predict(X_train)
col_names = list(Y.columns.values)

print(eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.998258   0.999330  0.998393  0.998862
request                 0.999027   0.998505  0.995826  0.997164
offer                   0.999898   1.000000  0.978947  0.989362
aid_related             0.998822   0.998777  0.998411  0.998594
medical_help            0.999590   1.000000  0.994888  0.997438
medical_products        0.999846   0.998991  0.997984  0.998487
search_and_rescue       0.999898   0.998249  0.998249  0.998249
security                0.999795   0.997101  0.991354  0.994220
military                0.999846   0.996865  0.998430  0.997647
water                   1.000000   1.000000  1.000000  1.000000
food                    0.999949   0.999545  1.000000  0.999773
shelter                 0.999898   1.000000  0.998847  0.999423
clothing                0.999898   0.996753  0.996753  0.996753
money                   1.000000   1.000000  1.000000  1.000000
missing_people          1.000000   1.000

In [8]:
# Calculate evaluation metrics for test set
Y_test_pred = pipe.predict(X_test)

eval_metrics0 = eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.820040   0.837362  0.948460  0.889455
request                 0.894575   0.835913  0.482143  0.611552
offer                   0.996465   0.000000  0.000000  0.000000
aid_related             0.772553   0.740088  0.689810  0.714065
medical_help            0.921777   0.575758  0.073218  0.129915
medical_products        0.954049   0.775000  0.096573  0.171745
search_and_rescue       0.977716   1.000000  0.052288  0.099379
security                0.980636   0.250000  0.008065  0.015625
military                0.966498   0.608696  0.062780  0.113821
water                   0.956201   0.870056  0.370192  0.519393
food                    0.936530   0.809145  0.562155  0.663407
shelter                 0.937606   0.814545  0.386874  0.524590
clothing                0.986476   0.764706  0.134021  0.228070
money                   0.979099   0.777778  0.049645  0.093333
missing_people          0.987245   0.000

In [9]:
# Calculation the proportion of each column that have label == 1
Y.sum()/len(Y)

related                   0.764792
request                   0.171892
offer                     0.004534
aid_related               0.417243
medical_help              0.080068
medical_products          0.050446
search_and_rescue         0.027816
security                  0.018096
military                  0.033041
water                     0.064239
food                      0.112302
shelter                   0.088904
clothing                  0.015560
money                     0.023206
missing_people            0.011449
refugees                  0.033618
death                     0.045874
other_aid                 0.132396
infrastructure_related    0.065506
transport                 0.046143
buildings                 0.051214
electricity               0.020440
tools                     0.006109
hospitals                 0.010873
shops                     0.004610
aid_centers               0.011872
other_infrastructure      0.044222
weather_related           0.280352
floods              

# 6. Improve your model
Use grid search to find better parameters.

In [10]:
# Define performance metric for use in grid search scoring object
def perform_metric(Y_T, Y_P):
    """Median F1 score for all classifiers
    
    inputs:
    Y_T: array. Array containing ArrayL labels.
    Y_P: array. Array containing ArrayP labels.
        
    Routputs:
    Median F1 score for all  classifiers
    """
    f1_list = []
    for i in range(np.shape(Y_P)[1]):
        f1 = f1_score(np.array(Y_T)[:, i], Y_P[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [11]:
# grid search

parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(perform_metric)
cv = GridSearchCV(pipe, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(74)
T_model = cv.fit(X_train, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5; 1/24] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 1/5; 1/24] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=0.138 total time= 1.4min
[CV 2/5; 1/24] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 2/5; 1/24] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=0.139 total time= 1.4min
[CV 3/5; 1/24] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 3/5; 1/24] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=0.141 total time= 1.4min
[CV 4/5; 1/24] START clf__estimator__min_samples_split=2, clf__estimator__n_estimat

[CV 5/5; 6/24] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5;, score=0.193 total time= 2.4min
[CV 1/5; 7/24] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1
[CV 1/5; 7/24] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1;, score=0.135 total time= 4.9min
[CV 2/5; 7/24] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1
[CV 2/5; 7/24] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1;, score=0.169 total time= 8.6min
[CV 3/5; 7/24] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1
[CV 3/5; 7/24] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1;, score=0.117 

[CV 4/5; 12/24] END clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5;, score=0.227 total time=  41.4s
[CV 5/5; 12/24] START clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5
[CV 5/5; 12/24] END clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5;, score=0.187 total time=  41.0s
[CV 1/5; 13/24] START clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1
[CV 1/5; 13/24] END clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1;, score=0.110 total time= 2.3min
[CV 2/5; 13/24] START clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1
[CV 2/5; 13/24] END clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1;, score=0.

[CV 3/5; 18/24] END clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5;, score=0.222 total time=  44.3s
[CV 4/5; 18/24] START clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5
[CV 4/5; 18/24] END clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5;, score=0.207 total time=  40.8s
[CV 5/5; 18/24] START clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5
[CV 5/5; 18/24] END clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5;, score=0.198 total time=  36.1s
[CV 1/5; 19/24] START clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1
[CV 1/5; 19/24] END clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1;, sc

[CV 2/5; 24/24] END clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5;, score=0.177 total time= 1.2min
[CV 3/5; 24/24] START clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5
[CV 3/5; 24/24] END clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5;, score=0.199 total time= 1.2min
[CV 4/5; 24/24] START clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5
[CV 4/5; 24/24] END clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5;, score=0.206 total time= 1.2min
[CV 5/5; 24/24] START clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5
[CV 5/5; 24/24] END clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5

In [12]:
#grid search results
T_model.cv_results_

{'mean_fit_time': array([  79.46295342,   54.15748191, 4230.2617157 ,  332.58128495,
         289.57834148,  133.44644918,  301.60594077,  111.73908267,
          54.75892372,   41.22895889,  166.77928853,   39.08115149,
         140.51004877,   86.38558602,  167.7642025 ,   84.17262964,
          48.86373706,   38.52895994,   46.11797876,   36.31747661,
         109.57557015,   73.51656733,  106.22781258,   67.85794935]),
 'std_fit_time': array([6.09273127e-01, 8.84806580e+00, 8.30257882e+03, 5.43139531e+02,
        6.42036292e+01, 2.26548970e+00, 1.05161857e+02, 3.71192751e+00,
        4.79075535e-01, 3.12951537e+00, 2.19287890e+02, 1.52811519e+00,
        3.91387077e+00, 1.02850773e+01, 7.31446192e+01, 5.01785618e+00,
        2.74865470e+00, 2.74435041e+00, 2.64018665e+00, 2.25928746e+00,
        5.17761681e+00, 7.58070245e-01, 3.70764721e+00, 8.28356362e-01]),
 'mean_score_time': array([3.08913784, 3.34390378, 3.65900145, 4.15753527, 5.7895504 ,
        4.83844686, 6.04107456, 4.20

In [13]:
# top mean score
np.max(T_model.cv_results_['mean_test_score'])

0.23607907126298913

In [14]:
# Parameters for top mean score
T_model.best_params_

{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 10,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

# 7. Test your model
Show the accuracy, precision, and recall of the tuned model.


Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
# evaluating metrics for test set
tuned_pred_test = T_model.predict(X_test)

eval_metrics1 = eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.809590   0.840519  0.926314  0.881333
request                 0.889811   0.769746  0.513393  0.615961
offer                   0.996465   0.000000  0.000000  0.000000
aid_related             0.755955   0.709084  0.690556  0.699697
medical_help            0.919010   0.469697  0.119461  0.190476
medical_products        0.955586   0.690476  0.180685  0.286420
search_and_rescue       0.977255   0.608696  0.091503  0.159091
security                0.980790   0.333333  0.008065  0.015748
military                0.967112   0.578947  0.147982  0.235714
water                   0.960350   0.808594  0.497596  0.616071
food                    0.938220   0.776632  0.624309  0.692190
shelter                 0.936991   0.766562  0.419689  0.542411
clothing                0.986937   0.730769  0.195876  0.308943
money                   0.979407   0.705882  0.085106  0.151899
missing_people          0.987245   0.333

In [16]:
# summary of 1st model
eval_metrics0.describe()


,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.945957,0.580300,0.204490,0.254393
std,0.052247,0.352629,0.271363,0.291971
min,0.772553,0.000000,0.000000,0.000000
25%,0.935915,0.350000,0.011845,0.023197
50%,0.956201,0.740088,0.057971,0.108844
75%,0.980636,0.836638,0.378533,0.521992
max,0.996465,1.000000,0.948460,0.889455


In [17]:
# summary of tuned model
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.945092,0.561930,0.247463,0.305620
std,0.055489,0.286398,0.268815,0.282367
min,0.755955,0.000000,0.000000,0.000000
25%,0.937606,0.409452,0.016808,0.032796
50%,0.960350,0.643678,0.123746,0.207283
75%,0.980944,0.766541,0.458643,0.579186
max,0.996465,1.000000,0.926314,0.881333


# 8. Try improving your model further. Here are a few ideas:
try other machine learning algorithms

add other features besides the TF-IDF

In [ ]:
# using SVM instead 
pipe2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [1, 2, 3],
              'clf__estimator__C':[1, 10, 100]}

cv2 = GridSearchCV(pipe2, param_grid = parameters2, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(77)
T_model2 = cv2.fit(X_train, Y_train)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5
[CV 1/5; 1/9] END clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5;, score=0.351 total time= 4.2min
[CV 2/5; 1/9] START clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5
[CV 2/5; 1/9] END clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5;, score=0.365 total time= 3.7min
[CV 3/5; 1/9] START clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5
[CV 3/5; 1/9] END clf__estimator__C=1, clf__estimator__degree=1, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5;, score=0.336 total time= 3.8min
[CV 4/5; 1/9] START clf__estimator__C=1, clf__estimator

In [ ]:
# grid search results
T_model2.cv_results_

median always zero therefore lets evaluate metrics

In [ ]:
# Calculate evaluation metrics for test set
tuned_pred_test2 = T_model2.predict(X_test)

eval_metrics2 = eval_metrics(np.array(Y_test), tuned_pred_test2, col_names)

print(eval_metrics2)

SVM doesnt work ,we will be using RANDOM FOREST FTW

# 9. Export your model as a pickle file

In [ ]:
# Pickle best model
pickle.dump(T_model, open('disaster_model.sav', 'wb'))

# 10. Use this notebook to complete train.py
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.